```python
import time

aa = time.time()
cnt = 0
set_index = new_df.index


for i in new_df.index:
  for j in (new_df.columns)[:17087]: # label 컬럼은 제외
    if j in set_index and j == i:
      new_df[j][i] = 1
      cnt += 1
    
print(f'총 {cnt}회 연산')
print(time.time() - aa,"초")
# 대략 3억 회 연산?
# 2^17087 인건가

# 하... 이거 언제 다되냐...
```

- 위 코드가 언제 끝날지 모르겠다...

In [2]:
pip install konlpy

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install jupyterthemes

In [4]:
import re
import pickle
import csv
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv("data/병명증상정보.csv", encoding='cp949')
dff = pd.read_csv("data/병명증상정보.csv", encoding='cp949')
df.head()

,담당자,부위,증상,질환명,진료과,TAG
0,박장훈,복부,신생아의 98%는 출생 후 24시간 내에 짙은 녹색의 태변을 배설합니다. 아기의 항...,선천성 거대결장증,소아외과,"히르쉬스프룽병 ,선청성대장질환 ,거대결장 ,대장조영술검사"
1,박장훈,복부,간에서 만들어진 담즙은 담도를 통해 소장으로 분비돼 소화를 돕는 데 사용됩니다. 그...,담도폐쇄증,"소화기내과, 간담췌외과, 소아외과","담도, 담도폐쇄증 ,billiary atresia ,간경변증 ,간이식 ,담도조영술 ..."
2,박장훈,복부,"조기 간암의 증상으로 피로감, 전신 쇠약감, 소화 불량, 상복부에 느껴지는 뻐근함,...",간암,"간담췌외과, 방사선종양학과, 혈액종양내과, 소화기내과, 간암센터","피로감, 구토 ,복통 ,황달 ,메스꺼움 ,식욕부진 ,체중감소 ,AFP ..."
3,박장훈,복부,"대표적인 증상으로는 복통, 설사, 체중 감소를 들 수 있습니다. 특히 10~20대 ...",크론병 Crohn disease,"소화기내과, 소아소화기영양과","복통 ,설사 ,식욕부진 ,체중감소 ,치루 ,장폐색 국소성 장염 ,궤양성 ..."
4,박장훈,복부,비만은 체내 근육량에 비해 지방조직이 과다한 상태를 의미합니다. 대한비만학회에서는 ...,고도비만 Severe obesity,"내분비내과, 가정의학과, 내분비당뇨병센터, 비만대사수술센터","빈혈, 속 쓰림 ,오심, 구토 ,복통 ,체중감소 ,위암개복수술 ,위암합병증 ..."


In [5]:
df

,담당자,부위,증상,질환명,진료과,TAG
0,박장훈,복부,신생아의 98%는 출생 후 24시간 내에 짙은 녹색의 태변을 배설합니다. 아기의 항...,선천성 거대결장증,소아외과,"히르쉬스프룽병 ,선청성대장질환 ,거대결장 ,대장조영술검사"
1,박장훈,복부,간에서 만들어진 담즙은 담도를 통해 소장으로 분비돼 소화를 돕는 데 사용됩니다. 그...,담도폐쇄증,"소화기내과, 간담췌외과, 소아외과","담도, 담도폐쇄증 ,billiary atresia ,간경변증 ,간이식 ,담도조영술 ..."
2,박장훈,복부,"조기 간암의 증상으로 피로감, 전신 쇠약감, 소화 불량, 상복부에 느껴지는 뻐근함,...",간암,"간담췌외과, 방사선종양학과, 혈액종양내과, 소화기내과, 간암센터","피로감, 구토 ,복통 ,황달 ,메스꺼움 ,식욕부진 ,체중감소 ,AFP ..."
3,박장훈,복부,"대표적인 증상으로는 복통, 설사, 체중 감소를 들 수 있습니다. 특히 10~20대 ...",크론병 Crohn disease,"소화기내과, 소아소화기영양과","복통 ,설사 ,식욕부진 ,체중감소 ,치루 ,장폐색 국소성 장염 ,궤양성 ..."
4,박장훈,복부,비만은 체내 근육량에 비해 지방조직이 과다한 상태를 의미합니다. 대한비만학회에서는 ...,고도비만 Severe obesity,"내분비내과, 가정의학과, 내분비당뇨병센터, 비만대사수술센터","빈혈, 속 쓰림 ,오심, 구토 ,복통 ,체중감소 ,위암개복수술 ,위암합병증 ..."
...,...,...,...,...,...,...
328,정의석,머리,"두개골이 완전히 닫히지 않은 2세 이하의 소아에게 수두증이 생기면, 뇌척수액이 축적...",수두증,신경외과,"큰 머리둘레, 대두증, 구토, 졸음, 두통, 발작, 눈이 아래로 처짐, 일몰 현상,..."
329,정의석,머리,시야가 어두워짐. 빛이 반짝거리거나 아지랑이가 피는 것 같은 시각 조짐. 두통. 전...,편두통,신경과,"박동성 두통, 관자놀이 통증, 오심, 구토, 어두운 시야, 아지랑이, 반짝거림, 조..."
330,정의석,머리,"발열, 피로감, 몸살 기운, 기침, 구토, 식욕 감소, 두통, 목이 뻣뻣함, 의식장...",바이러스 수막염,"감염내과, 신경과","발열, 피로, 목 뻣뻣함, 두통, 보챔, 기면, 졸음, 엔테로바이러스, 장바이러스,..."
331,정의석,머리,1~2주 잠복기. 대부분 무증상. 고열. 두통. 어지럼증. 구토. 설사. 의식 혼미...,일본뇌염,"감염내과, 신경과, 소아감염면역과, 소아신경과","고열, 두통, 복통, 설사, 경련, 모기, 사백신, 생백신"


In [6]:
import konlpy
from konlpy.tag import Okt


okt = Okt()

In [7]:
columns_list = [] # dataframe 만들 리스트
label = [] # 질환명 넣어둘 라벨 리스트

for i in df.index:
  # (df['질환명'][i])
  for j in okt.nouns( df['증상'][i]):
    # if "." in j or "," in j:
    #   print(j)
    columns_list.append(j)
    label.append(df['질환명'][i])

In [8]:
columns_list

['신생아',
 '출생',
 '후',
 '내',
 '녹색',
 '태변',
 '배설',
 '아기',
 '항문',
 '정상',
 '발달',
 '태변',
 '배설',
 '것',
 '선천성',
 '거대',
 '결장',
 '증',
 '의심',
 '수',
 '첫',
 '단서',
 '신생아',
 '시기',
 '이후',
 '평소',
 '배',
 '자주',
 '관장',
 '대변',
 '만성',
 '변비',
 '지속',
 '경우',
 '이',
 '병',
 '의심',
 '수',
 '신생아',
 '아기',
 '담즙',
 '구토',
 '팽창',
 '복부',
 '영양실조',
 '변비',
 '장',
 '등',
 '증상',
 '수',
 '소아',
 '청소년기',
 '치료',
 '만성',
 '변비',
 '진단',
 '기도',
 '간',
 '담즙',
 '통해',
 '소장',
 '분비',
 '소화',
 '데',
 '사용',
 '담즙',
 '분비',
 '못',
 '간',
 '정체',
 '간',
 '손상',
 '간경변증',
 '진행',
 '담도폐쇄증',
 '가장',
 '특징',
 '증상',
 '달',
 '안',
 '아기',
 '황달',
 '회색',
 '빛',
 '대변',
 '신생아',
 '황달',
 '아주',
 '증상',
 '모유',
 '수유',
 '해도',
 '때문',
 '자칫',
 '위험',
 '신호',
 '치기',
 '따라서',
 '기저귀',
 '아기',
 '변',
 '색깔',
 '확인',
 '것',
 '아기',
 '황달',
 '대변',
 '색깔',
 '황금색',
 '회색',
 '흰색',
 '점차',
 '즉시',
 '소아과',
 '소아',
 '외과',
 '전문의',
 '담도폐쇄',
 '여부',
 '확인',
 '조기',
 '간암',
 '증상',
 '피로',
 '감',
 '전신',
 '쇠약',
 '감',
 '소화',
 '불량',
 '상',
 '복부',
 '체중',
 '감소',
 '등',
 '꼽',
 '수',
 '증상',
 '간암',
 '암',
 '좀',
 '더',
 '진행',
 '간',
 '기능

In [9]:
len(columns_list), len(label) # 길이 같음

# label : 질환명을 columns_list 길이에 맞춰 담은 리스트

(17088, 17088)

In [10]:
set_list = set(columns_list)
len(set_list), len(columns_list) # 왤케 많이 줄어들지...?

(2614, 17088)

In [11]:
cnt_list = {} # 튜플 형태로 개수와 columns 항목을 함께 저장할 dictionary

for i in set_list:
  cnt_list[i] = columns_list.count(i)

cnt_list

{'건성': 1,
 '파킨슨병': 5,
 '쇠': 2,
 '치가': 1,
 '환경': 8,
 '특정': 16,
 '손': 37,
 '생체': 1,
 '날': 5,
 '피로': 34,
 '여자아이': 1,
 '자존감': 2,
 '과제': 1,
 '폐': 42,
 '원래': 1,
 '목소리': 11,
 '조현병': 5,
 '유지': 5,
 '작': 1,
 '단백질': 7,
 '양팔': 1,
 '습득': 1,
 '잠꼬대': 2,
 '악수': 2,
 '대상포진': 2,
 '화의': 1,
 '콧수염': 2,
 '주관': 1,
 '체트': 4,
 '콜리플라워': 2,
 '화성': 1,
 '심낭': 3,
 '일찍': 3,
 '수염': 12,
 '힘': 25,
 '중이염': 4,
 '신진대사': 4,
 '우울증': 10,
 '회로': 2,
 '조직': 17,
 '정신의학': 1,
 '영유': 1,
 '진단': 39,
 '세균': 10,
 '전파': 4,
 '다태임신': 1,
 '가을': 1,
 '노출': 20,
 '의존': 2,
 '심부전': 13,
 '구부리': 1,
 '부정맥': 8,
 '가렵기': 1,
 '졸림': 1,
 '굽': 1,
 '드뭅니': 1,
 '혈관염': 2,
 '금물': 1,
 '유산': 3,
 '물감': 5,
 '단태': 1,
 '차리기': 1,
 '사람': 88,
 '만': 6,
 '수주': 2,
 '서혜부': 3,
 '오': 2,
 '즐거움': 1,
 '중성': 1,
 '용혈': 3,
 '열': 32,
 '만큼': 4,
 '식중독': 1,
 '애인': 1,
 '당시': 1,
 '인체': 1,
 '대상': 1,
 '학교': 2,
 '피트': 1,
 '오한': 9,
 '광장공포증': 3,
 '일도': 3,
 '털': 5,
 '단어': 2,
 '식은땀': 1,
 '관리': 4,
 '반면': 6,
 '이동': 1,
 '좌우': 4,
 '조종사': 1,
 '항암': 1,
 '장점': 1,
 '직장': 8,
 '혀': 15,
 '외형': 1,
 '경계': 8

In [12]:
# cnt_list.sort(reverse = True,key = lambda x : x[1])
cnt_list.items() # set_list와 길이는 같다...

# 단순히 이렇게 비교하는건... 토큰화 하는것이랑 뭐가 다르지??
# 그래도 EDA처럼 숫자로 명시화 해놓으면 새로운 인사이트를 찾을 수 있지 않을까??

dict_items([('건성', 1), ('파킨슨병', 5), ('쇠', 2), ('치가', 1), ('환경', 8), ('특정', 16), ('손', 37), ('생체', 1), ('날', 5), ('피로', 34), ('여자아이', 1), ('자존감', 2), ('과제', 1), ('폐', 42), ('원래', 1), ('목소리', 11), ('조현병', 5), ('유지', 5), ('작', 1), ('단백질', 7), ('양팔', 1), ('습득', 1), ('잠꼬대', 2), ('악수', 2), ('대상포진', 2), ('화의', 1), ('콧수염', 2), ('주관', 1), ('체트', 4), ('콜리플라워', 2), ('화성', 1), ('심낭', 3), ('일찍', 3), ('수염', 12), ('힘', 25), ('중이염', 4), ('신진대사', 4), ('우울증', 10), ('회로', 2), ('조직', 17), ('정신의학', 1), ('영유', 1), ('진단', 39), ('세균', 10), ('전파', 4), ('다태임신', 1), ('가을', 1), ('노출', 20), ('의존', 2), ('심부전', 13), ('구부리', 1), ('부정맥', 8), ('가렵기', 1), ('졸림', 1), ('굽', 1), ('드뭅니', 1), ('혈관염', 2), ('금물', 1), ('유산', 3), ('물감', 5), ('단태', 1), ('차리기', 1), ('사람', 88), ('만', 6), ('수주', 2), ('서혜부', 3), ('오', 2), ('즐거움', 1), ('중성', 1), ('용혈', 3), ('열', 32), ('만큼', 4), ('식중독', 1), ('애인', 1), ('당시', 1), ('인체', 1), ('대상', 1), ('학교', 2), ('피트', 1), ('오한', 9), ('광장공포증', 3), ('일도', 3), ('털', 5), ('단어', 2), ('식은땀', 1), ('관리', 4), (

In [13]:
cnt_list

{'건성': 1,
 '파킨슨병': 5,
 '쇠': 2,
 '치가': 1,
 '환경': 8,
 '특정': 16,
 '손': 37,
 '생체': 1,
 '날': 5,
 '피로': 34,
 '여자아이': 1,
 '자존감': 2,
 '과제': 1,
 '폐': 42,
 '원래': 1,
 '목소리': 11,
 '조현병': 5,
 '유지': 5,
 '작': 1,
 '단백질': 7,
 '양팔': 1,
 '습득': 1,
 '잠꼬대': 2,
 '악수': 2,
 '대상포진': 2,
 '화의': 1,
 '콧수염': 2,
 '주관': 1,
 '체트': 4,
 '콜리플라워': 2,
 '화성': 1,
 '심낭': 3,
 '일찍': 3,
 '수염': 12,
 '힘': 25,
 '중이염': 4,
 '신진대사': 4,
 '우울증': 10,
 '회로': 2,
 '조직': 17,
 '정신의학': 1,
 '영유': 1,
 '진단': 39,
 '세균': 10,
 '전파': 4,
 '다태임신': 1,
 '가을': 1,
 '노출': 20,
 '의존': 2,
 '심부전': 13,
 '구부리': 1,
 '부정맥': 8,
 '가렵기': 1,
 '졸림': 1,
 '굽': 1,
 '드뭅니': 1,
 '혈관염': 2,
 '금물': 1,
 '유산': 3,
 '물감': 5,
 '단태': 1,
 '차리기': 1,
 '사람': 88,
 '만': 6,
 '수주': 2,
 '서혜부': 3,
 '오': 2,
 '즐거움': 1,
 '중성': 1,
 '용혈': 3,
 '열': 32,
 '만큼': 4,
 '식중독': 1,
 '애인': 1,
 '당시': 1,
 '인체': 1,
 '대상': 1,
 '학교': 2,
 '피트': 1,
 '오한': 9,
 '광장공포증': 3,
 '일도': 3,
 '털': 5,
 '단어': 2,
 '식은땀': 1,
 '관리': 4,
 '반면': 6,
 '이동': 1,
 '좌우': 4,
 '조종사': 1,
 '항암': 1,
 '장점': 1,
 '직장': 8,
 '혀': 15,
 '외형': 1,
 '경계': 8

In [14]:
list_for_new_df = [] # 중복 제거된 칼럼 데이터

for i in set_list:
  list_for_new_df.append(i)

In [15]:
new_df = pd.DataFrame(data = None, columns = list_for_new_df, index = df['질환명'])
new_df

,건성,파킨슨병,쇠,치가,환경,특정,손,생체,날,피로,...,피하지방,헛것,착상,반쪽,중독증,금세,덩어리,정면,포기,고여
질환명,,,,,,,,,,,,,,,,,,,,,
선천성 거대결장증,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
담도폐쇄증,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
간암,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
크론병 Crohn disease,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
고도비만 Severe obesity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
수두증,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
편두통,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
바이러스 수막염,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
new_df.columns

Index(['건성', '파킨슨병', '쇠', '치가', '환경', '특정', '손', '생체', '날', '피로',
       ...
       '피하지방', '헛것', '착상', '반쪽', '중독증', '금세', '덩어리', '정면', '포기', '고여'],
      dtype='object', length=2614)

In [17]:
new_df.to_csv("집가고싶다2.csv")

### 어구로 나타내기
- 이전 까지는 **단어** 위주로 평가하여 수식화 하려고 했는데,
- 현재 하나하나 모두 계산하는 방식으로 하려니 시간이 굉장히 많이 소요되고 있다..
- 어구로 나타내어 보며 기존에 보지 못했던 내용을 찾아보자...

In [18]:
tmp1 = []

for i in df.index:
  for j in okt.phrases( df['증상'][i]):
    tmp1.append(j)

In [19]:
len(tmp1)

16521

### 위에서 구한 데이터를 통해서,
- 크롤러 데이터와 유사도를 구해서 일정 점수 이상이 나온다면,
- 증상 데이터로 판단해도 되지 않을까?
- 자카드 유사도에서는 같은 단어에 한해 구하여 유사도를 구할 수 있다.
- 단순히 자주 나오는 단어가 핵심으로 판단되는 것이 아닌, 적절히 이를 평가해주는 모델이 TF-IDF 이다. 명일 (04/23) 에는 이를 시도해보자...

In [20]:
tmp1

['신생아',
 '신생아의 98%',
 '신생아의 98%는 출생',
 '신생아의 98%는 출생 후',
 '신생아의 98%는 출생 후 24시간',
 '신생아의 98%는 출생 후 24시간 내',
 '짙은 녹색',
 '짙은 녹색의 태변',
 '배설',
 '아기',
 '아기의 항문',
 '발달',
 '24시간',
 '태변',
 '못한다면 이것',
 '선천성',
 '선천성 거대결장증',
 '의심',
 '의심할 수',
 '의심할 수 있는 첫',
 '단서',
 '신생아 시기',
 '신생아 시기 이후',
 '평소',
 '평소 배',
 '자주',
 '관장',
 '대변',
 '만성',
 '만성 변비',
 '지속',
 '지속될 경우',
 '지속될 경우 이',
 '지속될 경우 이 병',
 '의심해볼 수',
 '영아기',
 '담즙',
 '있는 구토',
 '팽창',
 '팽창된 복부',
 '영양실조',
 '심한 변비',
 '등의 증상',
 '드물게는 소아',
 '드물게는 소아와 청소년기',
 '치료',
 '진단',
 '기도',
 '98%',
 '출생',
 '녹색',
 '항문',
 '이것',
 '거대',
 '결장',
 '시기',
 '이후',
 '변비',
 '경우',
 '구토',
 '복부',
 '증상',
 '소아',
 '청소년기',
 '담즙',
 '통해',
 '통해 소장',
 '분비',
 '소화',
 '데 사용',
 '정체',
 '간 손상',
 '간경변증',
 '진행',
 '담도폐쇄증',
 '담도폐쇄증의 가장',
 '증상',
 '한두',
 '한두 달',
 '안 된 어린 아기',
 '황달',
 '황달과 회색빛',
 '황달과 회색빛 대변',
 '신생아',
 '아주',
 '아주 흔한 증상',
 '모유',
 '모유 수유',
 '해도',
 '때문',
 '자칫',
 '자칫 위험',
 '자칫 위험 신호',
 '치기',
 '따라서',
 '따라서 기저귀',
 '아기',
 '아기의 변',
 '아기의 변 색깔',
 '확인',
 '확인하는 것',
 '대변',
 '대변 색깔',
 '황금색',
 '회색',
 '흰색

In [21]:
!pip install scikit-learn

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

# https://wikidocs.net/31698

corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]

tfidfv = TfidfVectorizer().fit(corpus)
print(tfidfv.transform(corpus).toarray())
print(tfidfv.vocabulary_)

[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


In [23]:
tfidfv = TfidfVectorizer().fit(tmp1)
print(tfidfv.transform(corpus).toarray())
print(tfidfv.vocabulary_)

# 이 방법은 아닌것 같다..

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
{'신생아': 2299, '신생아의': 2301, '98': 114, '출생': 3707, '24시간': 31, '짙은': 3604, '녹색': 859, '녹색의': 860, '태변': 3857, '배설': 1559, '아기': 2372, '아기의': 2375, '항문': 4082, '발달': 1489, '못한다면': 1368, '이것': 2908, '선천성': 1997, '거대결장증': 329, '의심': 2896, '의심할': 2898, '있는': 3056, '단서': 983, '시기': 2228, '이후': 2974, '평소': 3944, '자주': 3126, '관장': 561, '대변': 1042, '만성': 1258, '변비': 1602, '지속': 3521, '지속될': 3525, '경우': 417, '의심해볼': 2899, '영아기': 2609, '담즙': 1014, '구토': 622, '팽창': 3929, '팽창된': 3930, '복부': 1653, '영양실조': 2611, '심한': 2360, '등의': 1187, '증상': 3496, '드물게는': 1181, '소아': 2077, '소아와': 2082, '청소년기': 3657, '치료': 3740, '진단': 3550, '기도': 702, '거대': 328, '결장': 391, '통해': 3880, '소장': 2086, '분비': 1725, '소화': 2089, '사용': 1867, '정체': 3345, '손상': 2110, '간경변증': 242, '진행': 3566, '담도폐쇄증': 1010, '담도폐쇄증의': 1011, '가장': 219, '한두': 4070, '어린': 2515, '황달': 4258, '황달과': 4259, '회색빛': 4270, '아주': 2394, '흔한': 4317, '모유': 1342, '수유': 2155, '해도': 4095, '

In [ ]:
df

In [ ]:
tmp2 = []

for i in df['증상']:
    tmp2.append(i)

tmp2

In [ ]:
tfidfv = TfidfVectorizer().fit(tmp2)
print(tfidfv.transform(corpus).toarray())
print(tfidfv.vocabulary_)

In [ ]:
sorted(tfidfv.vocabulary_.items(), key=lambda x : x[1], reverse=True)

### 현재 상태
- 우선 현재 상태로는 증상을 구별하기 어려울 것 같다...
- 최소한 엇비슷하게라도 나와야 하는데 불필요한 단어 또한 순위가 높기 때문에 이대로는 사용할 수 없다.
- (추가) 생각해보니 위 데이터 (tmp2)는 중복된 데이터가 추가돼서 보다 부정확하게 나올 수 밖에 없었던 것 같다.

### TF-IDF의 의미
- TF-IDF(Term Frequency - Inverse Document Frequency)는 정보 검색과 텍스트 마이닝에서 이용하는 가중치로, 여러 문서로 이루어진 문서군이 있을 때 어떤 단어가 특정 문서 내에서 얼마나 중요한 것인지를 나타내는 통계적 수치이다. 문서의 핵심어를 추출하거나, 검색 엔진에서 검색 결과의 순위를 결정하거나, 문서들 사이의 비슷한 정도를 구하는 등의 용도로 사용할 수 있다.

- TF(단어 빈도, term frequency)는 특정한 단어가 문서 내에 얼마나 자주 등장하는지를 나타내는 값으로, 이 값이 높을수록 문서에서 중요하다고 생각할 수 있다. 하지만 단어 자체가 문서군 내에서 자주 사용되는 경우, 이것은 그 단어가 흔하게 등장한다는 것을 의미한다. 이것을 DF(문서 빈도, document frequency)라고 하며, 이 값의 역수를 IDF(역문서 빈도, inverse document frequency)라고 한다. TF-IDF는 TF와 IDF를 곱한 값이다.

- IDF 값은 문서군의 성격에 따라 결정된다. 예를 들어 '원자'라는 낱말은 일반적인 문서들 사이에서는 잘 나오지 않기 때문에 IDF 값이 높아지고 문서의 핵심어가 될 수 있지만, 원자에 대한 문서를 모아놓은 문서군의 경우 이 낱말은 상투어가 되어 각 문서들을 세분화하여 구분할 수 있는 다른 낱말들이 높은 가중치를 얻게 된다.

- 특정 문서 내에서 단어 빈도가 높을 수록, 그리고 전체 문서들 중 그 단어를 포함한 문서가 적을 수록 TF-IDF값이 높아진다. 따라서 이 값을 이용하면 모든 문서에 흔하게 나타나는 단어를 걸러내는 효과를 얻을 수 있다. IDF의 로그 함수 안의 값은 항상 1 이상이므로, IDF값과 TF-IDF값은 항상 0 이상이 된다. 특정 단어를 포함하는 문서들이 많을 수록 로그 함수 안의 값이 1에 가까워지게 되고, 이 경우 IDF값과 TF-IDF값은 0에 가까워지게 된다.

In [ ]:
tfidfv = TfidfVectorizer().fit(columns_list)
print(tfidfv.transform(corpus).toarray())
print(tfidfv.vocabulary_)

In [ ]:
sorted(tfidfv.vocabulary_.items(), key=lambda x : x[1], reverse=True)

# 이 상태에서도 뒤죽박죽이라 이대로는 사용하기 어렵다

### 그렇다면...
- 형용사 + 명사 형태로 나타날 때 를 기준으로 분별해보자

In [30]:
tmp = []

for i in df['증상']:
    tmp.append(i)    

In [31]:
a = ''.join(tmp)

In [ ]:
print(okt.pos(a))

### 그렇다면 어떤 형태로 구성된 어구를 골라야 할까?
- Adjective Noun Josa Noun, Noun Noun Josa Noun Noun, Noun Josa Verb Noun, Noun Josa Verb Adjective Noun, Punctuation Noun Punctuation, Adjective Noun Punctuation, Noun Verb, Noun Josa Noun, Noun Noun, Noun Noun Noun, Noun Noun Josa Verb Adjective, Noun Noun Josa Verb Noun, Noun Josa Verb, Adjective Noun Noun, Noun Noun Josa Noun
- etc...

In [ ]:
test = ["Adjective Noun Josa Noun", 
        "Noun Noun Josa Noun Noun", 
        "Noun Josa Verb Noun", 
        "Noun Josa Verb Adjective Noun", 
        "Punctuation Noun Punctuation", 
        "Adjective Noun Punctuation", 
        "Noun Verb", "Noun Noun", 
        "Noun Josa Noun", "Noun Noun Noun", "Noun Josa Verb", "Adjective Noun Noun", 
        "Noun Noun Josa Noun",
        "Noun Noun Josa Verb Adjective", 
        "Noun Noun Josa Verb Noun"]
tmp = [] 

for i in test:
    b = i.split(" ")
#     print(len(b)) 2, 3, 4, 5
    tmp.append(b)
tmp

In [34]:
from collections import deque

# 어구 개수에 따라 변수 설정
que2 = deque()

In [40]:
test_2 = deque()
words= deque()
find_word = []

for i in okt.pos(a):
    if len(test_2) < 2:
        test_2.append(i[1])
        words.append(i[0])
        if len(test_2) == 2:
            # print(list(test_2))
            # print(list(words))

            if list(test_2) in tmp:
                find_word.append(' '.join(words))
                
            test_2.popleft()
            words.popleft()

test_3 = deque()
words = deque()

for i in okt.pos(a):
    if len(test_3) < 3:
        test_3.append(i[1])
        words.append(i[0])
        if len(test_3) == 3:
            # print(list(test_2))
            # print(list(words))

            if list(test_3) in tmp:
                find_word.append(' '.join(words))
                
            test_3.popleft()
            words.popleft()

for i in okt.pos(a):
    if len(test_3) < 4:
        test_3.append(i[1])
        words.append(i[0])
        if len(test_3) == 4:
            # print(list(test_2))
            # print(list(words))

            if list(test_3) in tmp:
                find_word.append(' '.join(words))
                
            test_3.popleft()
            words.popleft()

for i in okt.pos(a):
    if len(test_3) < 5:
        test_3.append(i[1])
        words.append(i[0])
        if len(test_3) == 5:
            # print(list(test_2))
            # print(list(words))

            if list(test_3) in tmp:
                find_word.append(' '.join(words))
                
            test_3.popleft()
            words.popleft()
      
for i in okt.pos(a):
    if len(test_3) < 6:
        test_3.append(i[1])
        words.append(i[0])
        if len(test_3) == 6:
            # print(list(test_2))
            # print(list(words))

            if list(test_3) in tmp:
                find_word.append(' '.join(words))
                
            test_3.popleft()
            words.popleft()      


# for i in find_word[:10]:
#     print(i)

len(find_word)

17627

In [44]:
filename = "C:/Users/asd/2nd_project/jupyter_notebook/data/output.csv"
csv_file = open(filename, mode="w" ,newline='', encoding='utf-8')
csv_writer = csv.writer(csv_file)
for i in find_word:
    csv_writer.writerow([i])
csv_file.close()